In [2]:
import os
import time
import cv2
import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionV3, Xception
from tensorflow.keras.layers import Layer, Input, Embedding, GRU, Dense, Dropout, BatchNormalization, Concatenate
import tensorflow_hub as hub
from os import listdir
from tqdm import tqdm
from PIL import Image
import pandas as pd
from tensorflow.keras.models import load_model

def load_frames_from_directory(directory):
    img_id=[]
    img_path=[]
    for root, dirs, files in os.walk(directory):
        # temparary image and path
        ti=[]
        tp=[]
        for file in files:
            
            # Check if the file is an image file
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                frame_path = os.path.join(root, file)
                
                if file not in ti:
                    ti.append(file)
                    tp.append(frame_path)
                    
        if ti:
            img_id.append(ti)
            img_path.append(tp)
                    
    return img_id,img_path

All_image_id,All_image_path=load_frames_from_directory('Frames_Sentence_Level')

word_image_id,word_image_path=load_frames_from_directory('Frames_Word_Level')

All_image_id.extend(word_image_id)
All_image_path.extend(word_image_path)

df1=pd.read_csv('corpus_csv_files\\ISL Corpus sign glosses.csv')
df2=pd.read_excel('corpus_csv_files\\ISL_CSLRT_Corpus_frame_details.xlsx')
df3=pd.read_excel('corpus_csv_files\\ISL_CSLRT_Corpus_word_details.xlsx')

data_num=len(All_image_path)

captions = []

for i in range(len(All_image_path)):

    if 'ISL_CSLRT_Corpus\\' + All_image_path[i][0] in df2['Frames path'].values:
        word_value = df2.loc[df2['Frames path'] == 'ISL_CSLRT_Corpus\\' + All_image_path[i][0], 'Sentence'].values
        sign_gloss = df1.loc[df1['Sentence'] == word_value[0], 'SIGN GLOSSES'].values
        if sign_gloss.size<=0:
            sign_gloss=np.array([word_value[0].upper()])
        
    elif 'ISL_CSLRT_Corpus\\' + All_image_path[i][0] in df3['Frames path'].values:
        sign_gloss = df3.loc[df3['Frames path'] == 'ISL_CSLRT_Corpus\\' + All_image_path[i][0], 'Word'].values
        
    else:
        sign_gloss = np.array(["No caption found"])
        
    captions.append('<start> '+sign_gloss[0]+' <end>')
    

In [3]:
padded_frames=[]
max_length=30

for i in range(len(All_image_path)):
    if len(All_image_path[i])==max_length:
        frames=All_image_path[i]
        padded_frames.append(frames)
        continue
    if len(All_image_path[i])<max_length:
        frames=All_image_path[i] + [All_image_path[i][-1]]  * (max_length - len(All_image_path[i]))
        
    else:
        interval = len(All_image_path[i]) / max_length
        frames=[All_image_path[i][int(j)] for j in np.arange(0,len(All_image_path[i])-1,interval)]
        
    padded_frames.append(frames)

In [4]:
import mediapipe as mp
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [5]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [6]:
def draw_styled_landmarks(image, results):
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)) 

In [7]:
holistic=mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [8]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [9]:
def load_image(image_path):
    img=cv2.imread(image_path)
    img=cv2.resize(img,(800,800))
    image, results = mediapipe_detection(img, holistic)
    keypoints = extract_keypoints(results)
    return keypoints

In [10]:
tokenizer=tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~')
tokenizer.fit_on_texts(captions)
max_size=len(tokenizer.word_index)+1


train_seqs=tokenizer.texts_to_sequences(captions)

max_len=max(len(t) for t in train_seqs)

caption_vector=tf.keras.preprocessing.sequence.pad_sequences(train_seqs,padding='post')

In [11]:
#sequence,labels=[],[]
#for i in tqdm(range(len(padded_frames))):
    #sequence.append([load_image(img) for img in padded_frames[i]])
    #labels.append(caption_vector[i])
#sequence=np.array(sequence)

In [12]:

labels=np.array(captions)

In [13]:
sequence=np.load('seq2.npy')

In [14]:
#np.save('seq2.npy',sequence)

In [15]:
sequence.shape

(777, 30, 126)

In [16]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [17]:
X = sequence
y = labels

In [18]:
from sklearn.preprocessing import LabelEncoder


# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the labels to numerical values
y = label_encoder.fit_transform(captions)
y = to_categorical(y).astype(int)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [21]:
print(y_train.shape)
print(X_train.shape)

(621, 211)
(621, 30, 126)


In [22]:
#del model

In [25]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense , Dropout, Flatten

# Define the model
model = Sequential()

# Add LSTM layer with return_sequences=True for time-distributed behavior
model.add(LSTM(units=256, input_shape=(30, 126), return_sequences=True))
model.add(LSTM(units=256, return_sequences=False))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(units=250, activation='relu'))
model.add(Dense(units=250, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=211, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 30, 256)           392192    
                                                                 
 lstm_3 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 flatten_1 (Flatten)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 250)               64250     
                                                                 
 dense_4 (Dense)             (None, 250)               62750     
                                                                 
 dropout_3 (Dropout)         (None, 250)              

In [26]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=300, restore_best_weights=True)

In [27]:
model.fit(X_train, y_train, epochs=500,validation_split=0.1,callbacks=[early_stopping],batch_size=32)

Epoch 1/500
18/18 [==============================] - 14s 335ms/step - loss: 5.3684 - accuracy: 0.0090 - val_loss: 5.3352 - val_accuracy: 0.0000e+00
Epoch 2/500
18/18 [==============================] - 4s 251ms/step - loss: 5.2654 - accuracy: 0.0072 - val_loss: 5.2816 - val_accuracy: 0.0159
Epoch 3/500
18/18 [==============================] - 4s 243ms/step - loss: 5.1244 - accuracy: 0.0090 - val_loss: 5.3243 - val_accuracy: 0.0000e+00
Epoch 4/500
18/18 [==============================] - 4s 241ms/step - loss: 5.0605 - accuracy: 0.0179 - val_loss: 5.1914 - val_accuracy: 0.0000e+00
Epoch 5/500
18/18 [==============================] - 4s 245ms/step - loss: 4.9530 - accuracy: 0.0251 - val_loss: 5.2283 - val_accuracy: 0.0000e+00
Epoch 6/500
18/18 [==============================] - 4s 238ms/step - loss: 4.8949 - accuracy: 0.0179 - val_loss: 5.1366 - val_accuracy: 0.0317
Epoch 7/500
18/18 [==============================] - 4s 248ms/step - loss: 4.8019 - accuracy: 0.0197 - val_loss: 5.2457 - val

In [28]:
model.evaluate(X_test,y_test)

5/5 [==============================] - 0s 73ms/step - loss: 5.1515 - accuracy: 0.0000e+00


[5.151504039764404, 0.0]

In [29]:
model.save('model02.keras')